## Neural Networks Example

### Make sure that your are in the right directory

In [ ]:
##-----Assumes you are running in /scratch-----##
import os
os.chdir("/scratch/"+os.getenv("USER")+"/ML_with_Python")

### Load the packages

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers 

### Read the data

In [ ]:
import numpy as np

data_file = 'Data/cancer_data.csv'
target_file = 'Data/cancer_target.csv'
cancer_data=np.loadtxt(data_file,dtype=float,delimiter=',')
cancer_target=np.loadtxt(target_file, dtype=float, delimiter=',')


### Split the data

In [ ]:
from sklearn import model_selection

test_size = 0.30
seed = 7

train_data, test_data, train_target, test_target = model_selection.train_test_split(cancer_data,
        cancer_target, test_size=test_size, 
        random_state=seed)

### Convert the targets to "one-hot vectors"

In [ ]:
# Convert the classes to ‘one-hot’ vector 
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_target, num_classes=2) 
y_test = to_categorical(test_target, num_classes=2)

### Design the Model

In [ ]:
def define_model():
    model = keras.Sequential([ 
       layers.Dense(30, activation="relu"), 
       layers.Dense(2, activation="softmax") 
    ])
    model.compile(optimizer="rmsprop", 
              loss="binary_crossentropy",
              metrics=["accuracy"]
    )
    return(model)
    
model = define_model()

### Fit the Model

In [ ]:
num_epochs = 10
batch_size = 32
model.fit(train_data, y_train, epochs=num_epochs, batch_size=batch_size)

### Test the Model

In [ ]:
predictions = np.argmax(model.predict(test_data), axis=-1)
score = model.evaluate(test_data, y_test) 



### Display the Results

In [ ]:
from sklearn.metrics import confusion_matrix 

print('\nAccuracy:  %.3f' % score[1])

print(confusion_matrix(test_target, predictions))

#### Design distributed model

In [ ]:
from tensorflow import distribute
mirrored_strategy = distribute.MirroredStrategy(['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1'])

def define_distributed_model():
    with mirrored_strategy.scope():
        distributed_model = define_model()
    return distributed_model

distributed_model = define_distributed_model()

### Test performance of distributed model on data

In [ ]:
distributed_model.fit(train_data, y_train, epochs=num_epochs, batch_size=batch_size)

### Artificial augmentation of data to 1 million rows

In [ ]:
#Augmentation 
n = 1000000
m = cancer_data.shape[1]
aug_cancer_data = np.zeros([n,m])
for i in range(m):
    aug_cancer_data[:,i] += np.random.choice(cancer_data[:,i], size = n)
aug_cancer_target = np.argmax(model.predict(aug_cancer_data), axis=-1).reshape(n,)

#Split
train_data, test_data, train_target, test_target = model_selection.train_test_split(aug_cancer_data,
        aug_cancer_target, test_size=test_size, 
        random_state=seed)

# Convert the classes to ‘one-hot’ vector 
y_train = to_categorical(train_target, num_classes=2) 
y_test = to_categorical(test_target, num_classes=2)

#Adjust batch size
batch_size = int(.8*train_data.shape[0])

### Retrain undistributed model on Augmented data

In [ ]:

model = define_model()
model.fit(train_data, y_train, epochs=num_epochs, batch_size=batch_size)


### Retrain distributed model on Augmented data

In [ ]:

distributed_model = define_distributed_model()
distributed_model.fit(train_data, y_train, epochs=num_epochs, batch_size=batch_size)


### Activity: Try reducing the batch size from .8 to .2, what’s  effect does this have on relative performance?